# Models

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5-nano")
response = model.invoke("Why do parrots talk?")

print(response)

## 模型传入的参数

- model
- api_key
- temperature
- max_tokens
- max-retries



## 1. 调用

In [ ]:
response = model.invoke("Why do parrots have colorful feathers?")
print(response)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

conversation = [
    SystemMessage("You are a helpful assistant that translates English to French."),
    HumanMessage("Translate: I love programming."),
    AIMessage("J'adore la programmation."),
    HumanMessage("Translate: I love building applications.")
]

response = model.invoke(conversation)
print(response)  # AIMessage("J'adore créer des applications.")

### 流式传输

In [ ]:
for chunk in model.stream("Why do parrots have colorful feathers?"):
    print(chunk.text, end="|", flush=True)

In [ ]:
full = None  # None | AIMessageChunk
for chunk in model.stream("What color is the sky?"):
    full = chunk if full is None else full + chunk
    print(full.text)

# The
# The sky
# The sky is
# The sky is typically
# The sky is typically blue
# ...

print(full.content)
# Usually blue during the day. That blue comes from Rayleigh scattering: sunlight hits the atmosphere and the shorter blue wavelengths scatter in all directions. The color can change: red/orange/pink at sunrise and sunset, gray on cloudy days, and dark at night (with stars visible). Want the answer for a specific time or place?

### 批量 Batch

In [ ]:
# 返回整个批次的最终输出
responses = model.batch([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
])
for response in responses:
    print(response.content)

In [ ]:
# 逐个接收每个输入生成完成时的输出.在使用 batch_as_completed() 时，结果可能会乱序到达。每个结果都包含输入索引，以便在需要时匹配以重建原始顺序。
for response in model.batch_as_completed([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
]):
    print(response)

## 2.工具调用

In [ ]:
from langchain_core.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")

for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

## 2.1 工具执行循环

In [ ]:
# 将（可能多个）工具绑定到模型
model_with_tools = model.bind_tools([get_weather])

# Step 1: 模型生成工具调用
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: 执行工具并收集结果
for tool_call in ai_msg.tool_calls:
    # 使用生成的参数执行该工具
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: 将结果传回模型以获得最终响应
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

2.2 强制工具调用

强制使用任何工具： 
```python
model_with_tools = model.bind_tools([tool_1], tool_choice="any")
```

强制使用特定工具
```python
model_with_tools = model.bind_tools([tool_1], tool_choice="tool_1")
```

### 2.3 并行工具调用


In [ ]:
model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke(
    "What's the weather in Boston and Tokyo?"
)


# The model may generate multiple tool calls
print(response.tool_calls)
# [
#   {'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'call_1'},
#   {'name': 'get_time', 'args': {'location': 'Tokyo'}, 'id': 'call_2'}
# ]


# Execute all tools (can be done in parallel with async)
results = []
for tool_call in response.tool_calls:
    if tool_call['name'] == 'get_weather':
        result = get_weather.invoke(tool_call)
    # ...
    results.append(result)
    
print( results)

### 2.4 流式工具调用

In [ ]:
for chunk in model_with_tools.stream(
    "What's the weather in Boston and Tokyo?"
):
    # Tool call chunks arrive progressively
    if chunk.tool_call_chunks:
        for tool_chunk in chunk.tool_call_chunks:
            print(f"Tool: {tool_chunk.get('name', '')}")
            print(f"Args: {tool_chunk.get('args', '')}")

# Output:
# Tool: get_weather            # Loop 1
# Args:
# Tool:                        # Loop 2
# Args: {"loc
# Tool:                        # Loop 3
# Args: ation": "BOS"}
# Tool: get_time               # Loop 4
# Args:
# Args:
# Tool:                        # Loop 5
# Args: {"timezone": "Tokyo"}

In [ ]:
gathered = None
for chunk in model_with_tools.stream("What's the weather in Boston?"):
    gathered = chunk if gathered is None else gathered + chunk
    print(gathered)